In [1]:
include("../src/Utero.jl")
using .Utero 
using MLDatasets

In [ ]:
TrainData = MNIST(:train)
ValidData = MNIST(:test)

In [ ]:
densenet = Sequential(
    Dense(28*28, 16, acf=ReLU), 
    Dense(16, 16, acf=ReLU), 
    Dense(16, 16, acf=ReLU), 
    Dense(16, 10), 
    Softmax)

In [ ]:
predicted = []
@show size(TrainData.features)
X = reshape(TrainData.features, 60000, 28*28)
@show size(X)
predicted = zeros(60000, 10)
for (i, x) in enumerate(eachrow(X))
    predicted[i, :] = densenet(x)
end
@show size(predicted)
Accuracy(predicted, TrainData.targets)

In [ ]:
ctx = ⬅Context()
AddParams!(ctx, densenet)

# Perform gradient steps
for i in 1:10
    for (x, y) in DataIterator(TrainData.features, TrainData.targets)
        x = reshape(x, 28*28)
        y = OneHot(Int64(y+1), 10)
        (out, paramgrads) = ForwardBackward!(ctx, x -> MeanSquaredError(densenet(x), y), x)
        GradientStep!(0.0001, ctx.Params, paramgrads)
    end
    print("Iter Done")
end
Untrack!(densenet)

In [ ]:
predicted = []
@show size(TrainData.features)
X = reshape(TrainData.features, 28*28, 60000)
@show size(X)
probabilities = zeros(60000, 10)
for (i, x) in enumerate(eachcol(X))
    probabilities[i, :] = densenet(x)
end
Accuracy(probabilities, TrainData.targets.+1)
